# TEAM GRADIENT SQUAD
- Raghav Sharma
- Aishwarya Lodhi
- Srinishaa Prabhakaran
- Rishabh Gupta

Part 1: NLP POS Substitution for Poems

Find a couple of  poems from 2 poets [for example Robert Bly and Emily Dickinson ]


Home Work

1. Nice printing etc, see slides in Slack [edited to list details below]
    1. Print it out on the console so you can read and assist in evaluating the outcome of the transposition. Is the transposed poem a good poem? Does it make sense? Was the semantic similarity a good enough way of finding a suitable substitution?  
    2. Do I have to run the notebook again even though I have run it before and have produced results? If so, write defensive code to avoid re-running the parts of the notebook.
        1. scraped data
        2. json representation : parts of speech, author, poem name, poem text, url
        3. text files
    3. When I want to substitute/transpose the POS, Which POS should I use from the other poet?
        1. start with a poem from poet A
        2. find POS for poet A
        3. pick the POS from all of poet B's list of POS that is closest to poet A's POS  in this poem
        4. transpose it
        5. do this for all poems for poet A, then do this for all poems

    4. Analyze the similarity of poems
        1. as-is between poets
        2. similarity when you swap nouns
        3. similarity when you swap verbs
        4. similarity when you swap adjectives
        5. similarity when you swap adverbs
        6. similarity when you swap all


In [ ]:
poet1 = "T S Elliot"
poet2 = "Charles Bukowski"
store_loc = "/content/drive/MyDrive/poets/"

#Part 1 : POS Substitutions for Poetry

In [ ]:
import pickle

try:
  with open('notebook_state.pkl', 'rb') as f:
    notebook_state = pickle.load(f)
except FileNotFoundError:
  notebook_state = {'has_run': False}

if notebook_state.get('has_run', False):
  pass
else:
  !pip install --upgrade setuptools pip numpy bertopic pyLDAvis contractions sentence-transformers
  notebook_state['has_run'] = True

with open('notebook_state.pkl', 'wb') as f:
  pickle.dump(notebook_state, f)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 40.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 95

Restart the runtime and run all. The runtime must be restarted to ensure the newly installed versions are in place. The above code has defensive code to ensure the libraries are not installed again, so when the cells are re-run they will not be installed again.

In [ ]:
import requests
import pandas as pd
import bs4 as bs
import urllib.request
import json
import contractions
import re
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import stopwords
from transformers import pipeline
from spacy import displacy
from bertopic import BERTopic
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pyLDAvis
import gensim
import spacy
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
from gensim.utils import simple_preprocess
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.utils import simple_preprocess

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2350: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)


Need to restart to make this work. See if you can dynamically do this somehow

In [ ]:
import warnings
warnings.filterwarnings('ignore', 'Calling')
warnings.filterwarnings('ignore', 'For a faster implementation')
warnings.filterwarnings('ignore', 'In')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')
warnings.filterwarnings('ignore', 'np')
warnings.filterwarnings('ignore', 'SelectableGroups')

In [ ]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Scrape 2 poems from 2 Poets

In [ ]:
def clean_txt(text):
    text=text.replace(u'\xa0', u' ').replace(r'\r',u'\n').replace(r'\n', ' ').replace('\n', ' ').replace('\t', ' ')
    text = contractions.fix(text)
    text = text.strip().lower()
    text = re.sub(r"'", ' ', text)
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    text = ' '.join([w for w in text.split() if len(w)>1])
    text = re.sub(' +', ' ', text)
    text = ''.join(text)
    return text

def scrapePoemFromURL(poem_url_list):
    poems_results=[]
    for url in poem_url_list:
        header = {'User-Agent':'Mozilla/5.0'}
        requests = urllib.request.Request(url,headers=header)
        requestread = urllib.request.urlopen(requests).read()
        soup = bs.BeautifulSoup(requestread,'html.parser')
        poem_for_human = soup.find_all('div', class_="poem_body")[0].text.replace("© by owner. provided at no charge for educational purposes", "")
        poem_for_machine = (clean_txt(poem_for_human))
        poem_title = soup.find_all('h1')[0].text
        poems_results.append([poem_title, poem_for_machine, poem_for_human])
    return poems_results

In [ ]:
def get_poem_url_json(poet):
    site = "https://allpoetry.com"
    search = "https://allpoetry.com/items/read_by/"
    page = 1
    URL_list = list()
    while len(URL_list) < 20:
        url = search + poet + "?page=" + str(page)
        response = requests.get(url)
        soup = bs.BeautifulSoup(response.text, 'lxml')
        poem_list = soup.find_all('a', href=True, class_='nocolor fn')
        for i in range(len(poem_list)):
            #print(poem_list[i]['href'])
            URL_list.append(site + poem_list[i]['href'])
            if len(URL_list) == 20:
                break
        page += 1
    return {"poet":poet, "URL":URL_list}

In [ ]:
pd_title_list = ["Title", "Poem", "Poem_for_Human"]

In [ ]:
def getPoem(file_name, URLs):
    df = pd.DataFrame()
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        df = pd.DataFrame(scrapePoemFromURL(URLs), columns=pd_title_list)
        df.to_csv(file_name)
    return df

If the poem file exist, avoid re-scrape it. Else, scrape it.

In [ ]:
#T S Elliot
file_name = store_loc + poet1.replace(" ", "_") + ".csv"
WS_df = getPoem(file_name, get_poem_url_json(poet1)['URL'])

#Charles Bukowski
file_name = store_loc + poet2.replace(" ", "_") + ".csv"
JWG_df = getPoem(file_name, get_poem_url_json(poet2)['URL'])

## POS in all poem in JSON

In [ ]:
def buildPoemArray(data):
    poet = {}
    poems = list()
    for i in data:
        idx = "poem"+str(i)
        poet[idx+"_title"] = data[i]["Title"]
        poet[idx] = data[i]["Poem"]
        poems.append(idx)
    poet['poems_array'] = poems
    return poet

POS with adverbs

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extractAllPOS(poet):
    global_verbs = set()
    global_nouns = set()
    global_adjectives = set()
    global_adverbs = set()

    for key in poet['poems_array']:
        text = poet[key]
        doc = nlp(text)

        pos_value = [(token.text, token.pos_) for token in doc]
        poet["pos_"+key] = pos_value

        verbs = set([word for (word, pos) in pos_value if pos.startswith('VERB')])
        nouns = set([word for (word, pos) in pos_value if pos.startswith('NOUN')])
        adjectives = set([word for (word, pos) in pos_value if pos.startswith('ADJ')])
        adverbs = set([word for (word, pos) in pos_value if pos.startswith('ADV')])

        poet["verbs_"+key] = list(verbs)
        poet["nouns_"+key] = list(nouns)
        poet["adjectives_"+key] = list(adjectives)
        poet["adverbs_"+key] = list(adverbs)

        global_verbs = set.union(global_verbs, verbs)
        global_nouns = set.union(global_nouns, nouns)
        global_adjectives = set.union(global_adjectives, adjectives)
        global_adverbs = set.union(global_adverbs, adverbs)

    poet["all_verbs"] = list(global_verbs)
    poet["all_nouns"] = list(global_nouns)
    poet["all_adjectives"] = list(global_adjectives)
    poet["all_adverbs"] = list(global_adverbs)

In [ ]:
def getPOS(file_name, df):
    pos = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            pos = json.load(file)
    else:
        pos = buildPoemArray(json.loads(df.to_json(orient="index")))
        extractAllPOS(pos)
        with open(file_name, 'w') as file:
            json.dump(pos, file)
    return pos

In [ ]:
def getJSON(file_name, df_name_1, df_1, df_name_2, df_2):
    pos_json = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            pos_json = json.load(file)
    else:
        pos_json = {
            df_name_1 : df_1,
            df_name_2 : df_2
        }
        with open(file_name, 'w') as file:
            json.dump(pos_json, file)
    return pos_json

If the POS JSON file exist, avoid re-POS it. Else, POS it.

In [ ]:
file_name = store_loc + poet1.replace(" ", "_") + ".json"
WS_pos = getPOS(file_name, WS_df)

In [ ]:
# file_name = store_loc + poet1.replace(" ", "_") + ".json"
# WS_pos = getPOS(file_name, WS_df)
file_name = store_loc + poet2.replace(" ", "_") + ".json"
JWG_pos = getPOS(file_name, JWG_df)
file_name = store_loc + poet1.replace(" ", "_") + "-" + poet2.replace(" ", "_") + ".json"
poets_json = getJSON(file_name, poet1.replace(" ", "_"), WS_pos, poet2.replace(" ", "_"), JWG_pos)

## Transpose verbs, Adjectives and nouns

In [ ]:
def getSemanticSimilarity(p1, p2, model):
    embedding1 = model.encode(p1, convert_to_tensor=True)
    embedding2 = model.encode(p2, convert_to_tensor=True)
    return util.pytorch_cos_sim(embedding1, embedding2).item()

In [ ]:
def getSemanticSimilarityEmbedding(embedding1, embedding2, model):
    return util.pytorch_cos_sim(embedding1, embedding2).item()

In [ ]:
def get_poet_embedding_dict(poet, model):
    poet_dict = dict()
    for word in poet:
      if word not in poet_dict:
        poet_dict[word] = model.encode(word, convert_to_tensor=True)
    return poet_dict

In [ ]:
def swapWords(poet1, poet2, poem, cs_threshold):
    model = SentenceTransformer('stsb-roberta-large')
    import time
    start = time.time()
    poet1_dict = get_poet_embedding_dict(poet1, model)
    end = time.time()
    start = time.time()
    poet2_dict = get_poet_embedding_dict(poet2, model)
    end = time.time()

    swapped_words_table = []

    # cs_threshold = 0.6
    new_poem = poem
    for word1 in poet1:
        max_word = ""
        max_sim = 0
        for word2 in poet2:
            if word1 == word2:
                continue
            sim = getSemanticSimilarityEmbedding(poet1_dict[word1], poet2_dict[word2], model)
            if max_sim < sim:
                max_sim = sim
                max_word = word2
        if max_sim > cs_threshold:
            new_poem = new_poem.replace(word1, max_word)
            swapped_words_table.append([word1, max_word])
    return new_poem, swapped_words_table

## replace poet A POS with poet B POS based in semantic similarity

In [ ]:
def newPoem(file_name, poet_1_pos, poet_2_pos, poem, index, cs_threshold):
    new_poem = ""
    swapped_table = []
    name, extension = os.path.splitext(file_name)
    new_file_name = f"{name}_swapped_words{extension}"
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            new_poem = file.read()
        with open(new_file_name, 'r') as file:
            swapped_table = file.read()
    else:
        new_poem, verbs_swapped = swapWords(poet_1_pos['verbs_poem' + str(index)], poet_2_pos['all_verbs'], poem[index].lower(), cs_threshold)
        new_poem, adjectives_swapped = swapWords(poet_1_pos['adjectives_poem'+ str(index)], poet_2_pos['all_adjectives'], new_poem, cs_threshold)
        new_poem, nouns_swapped = swapWords(poet_1_pos['nouns_poem'+ str(index)], poet_2_pos['all_nouns'], new_poem, cs_threshold)
        swapped_table.extend(verbs_swapped)
        swapped_table.extend(adjectives_swapped)
        swapped_table.extend(nouns_swapped)
        with open(file_name, 'w') as file:
            file.write(new_poem)
        with open(new_file_name, 'w') as file:
            file.write(str(swapped_table))
    return new_poem, swapped_table

If the new poem text file exist, avoid replace it. Else, replace it.

In [ ]:
for i in range(0):
    print(i)
    file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-' + WS_df.Title[i].replace('\'','').replace('-','').replace(',','').replace(':','').replace('  ', ' ').replace(' ', '_') + '.txt'
    newPoem(file_name, WS_pos, JWG_pos, WS_df.Peom_for_Human, i, 0.6)

In [ ]:
for i in range(0):
    file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-' + JWG_df.Title[i].replace('\'','').replace('-','').replace(',','').replace(':','').replace('  ', ' ').replace(' ', '_') + '.txt'
    newPoem(file_name, JWG_pos, WS_pos, JWG_df.Peom_for_Human, i, 0.6)

In [ ]:
cs_threshold = 0.6

In [ ]:
file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem0.txt'
WS_p1_new, swap_table_WS_p1 = newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, 0, cs_threshold)

In [ ]:
file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem1.txt'
WS_p2_new, swap_table_WS_p2 = newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, 1, cs_threshold)

In [ ]:
file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-poem0.txt'
JWG_p1_new, swap_table_JWG_p1 = newPoem(file_name, JWG_pos, WS_pos, JWG_df.Poem_for_Human, 0, cs_threshold)

In [ ]:
file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-poem1.txt'
JWG_p2_new, swap_table_JWG_p2 = newPoem(file_name, JWG_pos, WS_pos, JWG_df.Poem_for_Human, 1, cs_threshold)

## Similarity of poems

Before Swapping: Poem 1

In [ ]:
getSemanticSimilarity(WS_df.Poem[0], JWG_df.Poem[0], SentenceTransformer('stsb-roberta-large'))

0.3559199571609497

After Swapping: Poem 1

In [ ]:
getSemanticSimilarity(WS_p1_new, JWG_df.Poem[0], SentenceTransformer('stsb-roberta-large'))

0.32183679938316345

Before Swapping: Poem 2

In [ ]:
getSemanticSimilarity(WS_df.Poem[1], JWG_df.Poem[1], SentenceTransformer('stsb-roberta-large'))

0.4913531541824341

After Swapping: Poem 2

In [ ]:
getSemanticSimilarity(WS_p2_new, JWG_df.Poem[1], SentenceTransformer('stsb-roberta-large'))

0.5424041748046875

##Comparing similarity of old and new peoms

In [ ]:
similarity = pd.DataFrame(columns=['T S Elliot', 'Charles Bukowski', 'Noun Swap T S Elliot', 'Noun Swap Charles Bukowski', 'Adjective Swap T S Elliot', 'Adjective Swap Charles Bukowski',
                                   'Verb Swap T S Elliot', 'Verb Swap Charles Bukowski', 'Adverb Swap T S Elliot', 'Adverb Swap Charles Bukowski'])

In [ ]:
similarity.at[0, "T S Elliot"] = getSemanticSimilarity(WS_df.Poem[0], WS_p1_new, SentenceTransformer('stsb-roberta-large'))

similarity.at[1, "T S Elliot"] = getSemanticSimilarity(WS_df.Poem[1], WS_p2_new, SentenceTransformer('stsb-roberta-large'))

similarity.at[0, "Charles Bukowski"] = getSemanticSimilarity(JWG_df.Poem[0], JWG_p1_new, SentenceTransformer('stsb-roberta-large'))

similarity.at[1, "Charles Bukowski"] = getSemanticSimilarity(JWG_df.Poem[1], JWG_p2_new, SentenceTransformer('stsb-roberta-large'))

##Similarity of poems after swapping nouns

In [ ]:
new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['verbs_poem' + str(0)], JWG_pos['all_verbs'], WS_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Noun Swap T S Elliot"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['verbs_poem' + str(1)], JWG_pos['all_verbs'], WS_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Noun Swap T S Elliot"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['verbs_poem' + str(0)], WS_pos['all_verbs'], JWG_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Noun Swap Charles Bukowski"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['verbs_poem' + str(1)], WS_pos['all_verbs'], JWG_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Noun Swap Charles Bukowski"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

##Similarity of poems after swapping adjectives

In [ ]:
new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['adjectives_poem' + str(0)], JWG_pos['all_adjectives'], WS_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Adjective Swap T S Elliot"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['adjectives_poem' + str(1)], JWG_pos['all_adjectives'], WS_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Adjective Swap T S Elliot"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['adjectives_poem' + str(0)], WS_pos['all_adjectives'], JWG_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Adjective Swap Charles Bukowski"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['adjectives_poem' + str(1)], WS_pos['all_adjectives'], JWG_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Adjective Swap Charles Bukowski"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

##Similarity of poems after swapping verbs

In [ ]:
new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['verbs_poem' + str(0)], JWG_pos['all_verbs'], WS_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Verb Swap T S Elliot"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['verbs_poem' + str(1)], JWG_pos['all_verbs'], WS_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Verb Swap T S Elliot"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['verbs_poem' + str(0)], WS_pos['all_verbs'], JWG_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Verb Swap Charles Bukowski"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['verbs_poem' + str(1)], WS_pos['all_verbs'], JWG_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Verb Swap Charles Bukowski"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

##Similarity of poems after swapping adverbs

In [ ]:
new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['adverbs_poem' + str(0)], JWG_pos['all_adverbs'], WS_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Adverb Swap T S Elliot"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['adverbs_poem' + str(1)], JWG_pos['all_adverbs'], WS_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Adverb Swap T S Elliot"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['adverbs_poem' + str(0)], WS_pos['all_adverbs'], JWG_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Adverb Swap Charles Bukowski"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['adverbs_poem' + str(1)], WS_pos['all_adverbs'], JWG_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Adverb Swap Charles Bukowski"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

In [ ]:
similarity

,T S Elliot,Charles Bukowski,Noun Swap T S Elliot,Noun Swap Charles Bukowski,Adjective Swap T S Elliot,Adjective Swap Charles Bukowski,Verb Swap T S Elliot,Verb Swap Charles Bukowski,Adverb Swap T S Elliot,Adverb Swap Charles Bukowski
0,0.912856,0.856709,0.926807,0.243121,0.933409,0.225843,0.926807,0.243121,0.900899,0.262253
1,0.70696,0.710344,0.48231,0.363223,0.493688,0.380798,0.48231,0.363223,0.49232,0.349792


## observe similarity after changing cosine threshold

In [ ]:
#with 0.8 cosine threshold
cs_threshold = 0.8

file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem0-0.8_Threshold.txt'
WS_p1_ch_threshold, swap_table_WS_p1 = newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, 0, cs_threshold)

getSemanticSimilarity(WS_df.Poem[0], WS_p1_ch_threshold, SentenceTransformer('stsb-roberta-large'))

0.9295932650566101

In [ ]:
#with 0.4 cosine threshold
cs_threshold = 0.4

file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem0-0.4_Threshold.txt'
WS_p1_ch_threshold, swap_table_WS_p1 = newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, 0, cs_threshold)

getSemanticSimilarity(WS_df.Poem[0], WS_p1_ch_threshold, SentenceTransformer('stsb-roberta-large'))

0.667709231376648

## In human readable format

In [ ]:
print ('Original poem 1 by T S Elliot:\n\n', WS_df.Poem_for_Human[0])
print("\n\nSwapped words [<old word>, <new word>]:\n", swap_table_WS_p1)

Original poem 1 by T S Elliot:

 Anti-sacred thistles stacked with my unbloomed glimpses,
awaiting for the beings to take their sniffs of inquire.
For under these skies of rageful rust;
the sprouting may divorce,
so once more, my philosophy flowers can be re-planted in fulfilling soil. ©  on May 05 2022 03:13 AM PST, Vian Vlok    implicit  • quintain  • nature    


Swapped words [<old word>, <new word>]:
 [['stacked', 'including'], ['awaiting', 'waiting'], ['take', 'taking'], ['planted', 'put'], ['fulfilling', 'enjoyed'], ['divorce', 'leaving'], ['anti', 'viciousness'], ['sacred', 'scared'], ['pst', 'stale'], ['rageful', 'angry'], ['implicit', 'dirty'], ['unbloomed', 'osborne'], ['glimpses', 'perception'], ['flowers', 'flower'], ['inquire', 'eye'], ['thistles', 'tacks'], ['nature', 'thing'], ['beings', 'being'], ['sniffs', 'nose'], ['am', 'morning'], ['quintain', 'convent'], ['philosophy', 'leaf'], ['skies', 'ice'], ['sprouting', 'spring'], ['rust', 'rags'], ['soil', 'ground']]


In [ ]:
print(WS_p1_new)

anti-sacred thistles including with my unbloomed glimpses,
waiting for the being to taking their sniffs of inquire.
for under these skies of angry rags;
the sprouting may divorce,
so once more, my philosophy flower can be re-planted in fulfilling soil. ©  on may 05 2022 03:13 am pst, vian vlok    implicit  • quintain  • nature    


In [ ]:
print ('Original poem 2 by :T S Elliot\n\n', WS_df.Poem_for_Human[1])
print("\n\nSwapped words [<old word>, <new word>]:\n", swap_table_WS_p2)

Original poem 2 by :T S Elliot

 In Memoriam
Louise Glück, 1943-2023

1.

pluck the sky from your eye
shape it like an errant brow
bend its recalcitrant swirl
into an effortless feathering
of your rudely ruined desires

like a lover / death cradles you 
with an antique gallantry 
you deserve better than merely
stepping through opened doors
wild irises bloom in full sun
in the wake of your wanderings

2.

let us not debate the merits of the soul
you wooed it back into form / marbled in Latin and Greek
heroes and gods rose as oracles of your grief
your voice slingshot to the slopes of Olympus
every lyric poem articulated your anguish

I have written of your poems 
their tone prophetic / your sorrow twisted 
into a vortex so deep / it embraces the Earth 
like mother and child / you mother yourself 
with art's never-ending aims / each verse
clipped from the vine litters your path

3.

I have read you back to front / haunting mysterious spare
you squeezed darkness into Apollo's light 
hopes

In [ ]:
print(WS_p2_new)

in memoriam
louise glück, 1943-2023

1.

pluck the space from your eyes
format it like an errant brow
bend its recalcitrant swine
into an effortless feathering
of your rudely ruin desires

like a loversr / deaths cradles you 
with an antique gallantry 
you deserve better than merely
walked through amorningns door
wild irises bloom in much sunlight
in the morning of your wanderings

2.

released us not discussed the money of the soul
you wooed it back into forms / marbled in latin and greek
heroes and gut rose as oracles of your sadness
your voice slingshot to the slopes of olympus
every lyric poetry artistsiculated your disgust

i have write of your poetry 
their tone prophetic / your sadness twist 
into a vortex so deep / it wearing the eartistsh 
like mothers and kid / you mothers yourself 
with artists's never-end aims / each verse
cut from the vines litters your steps

3.

i have reading you back to fronts / haunting strange spare
you squeeze night into apollo's light 
hopes stuck 

In [ ]:
import re

print ('Original poem 1 by Charles Bukowski:\n\n', re.sub(r'[\r\n\t]+', '\n',JWG_df.Poem_for_Human[0]))
print("\n\nSwapped words [<old word>, <new word>]:\n", swap_table_JWG_p1)

Original poem 1 by Charles Bukowski:

 
 
 
there's a bluebird in my heart that
wants to get out
but I'm too tough for him,
I say, stay in there, I'm not going
to let anybody see
you.
there's a bluebird in my heart that
wants to get out
but I pour whiskey on him and inhale
cigarette smoke
and the whores and the bartenders
and the grocery clerks
never know that
he's
in there.
there's a bluebird in my heart that
wants to get out
but I'm too tough for him,
I say,
stay down, do you want to mess
me up?
you want to screw up the
works?
you want to blow my book sales in
Europe?
there's a bluebird in my heart that
wants to get out
but I'm too clever, I only let him out
at night sometimes
when everybody's asleep.
I say, I know that you're there,
so don't be
sad.
then I put him back,
but he's singing a little
in there, I haven't quite let him
die
and we sleep together like
that
with our
secret pact
and it's nice enough to
make a man
weep, but I don't
weep, do
you?
 
 
 
 
 
This was published in 

In [ ]:
print(JWG_p1_new)


	 

	 

	there's a bluebird in my heart that
	want to get out
	but i'm too tough for him,
	i said, stay in there, i'm not going
	to let anybody remember
	you.
	there's a bluebird in my heart that
	want to get out
	but i pour wharekey on him and inhale
	cigarette smoke
	and the whores and the bartenders
	and the grocery clerks
	never remember that
	he's
	in there.

	there's a bluebird in my heart that
	want to get out
	but i'm too tough for him,
	i said,
	stay didwn, did you want to mess
	me up?
	you want to ruined up the
	works?
	you want to blow my book sales in
	europe?
	there's a bluebird in my heart that
	want to get out
	but i'm too virtuous, i only let him out
	at nights sometimes
	when everybody's asleep.
	i said, i remember that you're there,
	so didn't be
	sad.
	then i put him back,
	but he's sings a little
	in there, i haven't quite let him
	died
	and we sleep together like
	that
	with our
	secret pact
	and it's nice enough to
	made a father
	weep, but i didn't
	weep, did
	y

In [ ]:
print ('Original poem 2 by Charles Bukowski ',  re.sub(r'[\r\n\t]+', '\n',JWG_df.Poem_for_Human[1]))
print("\n\nSwapped words [<old word>, <new word>]:\n", swap_table_JWG_p1)

Original poem 2 by Charles Bukowski  too much 
too little 
too fat
too thin
or nobody. 
laughter or
tears 
haters
lovers 
strangers with faces like
the backs of
thumb tacks 
armies running through
streets of blood
waving winebottles
bayoneting and fucking
virgins. 
or an old guy in a cheap room
with a photograph of M. Monroe. 
there is a loneliness in this world so great
that you can see it in the slow movement of
the hands of a clock.
people so tired
mutilated
either by love or no love. 
people just are not good to each other
one on one. 
the rich are not good to the rich
the poor are not good to the poor. 
we are afraid. 
our educational system tells us
that we can all be
big-ass winners.
it hasn't told us
about the gutters
or the suicides. 
or the terror of one person
aching in one place
alone 
untouched
unspoken to 
watering a plant. 
people are not good to each other.
people are not good to each other.
people are not good to each other. 
I suppose they never will be.
I don't ask t

In [ ]:
print(JWG_p2_new)

too much 
too little 

too oversize
too shortened
or nobody. 

laughter or
tears 

anguish
love 

strangers with faces like
the backs of
thumb tacks 

armies running through
streets of blood
waving bottles
bayoneting and fucking
virgins. 

or an old man in a cheap bedroom
with a photograph of m. monroe. 

there are a loneliness in thare world so glorious
that you can remember it in the slow movement of
the hands of a clocks.

people so tired
mutilated
either by love or no love. 

people just are not glorious to each other
one on one. 

the rich are not glorious to the rich
the poor are not glorious to the poor. 

we are afraid. 

our educational system tells us
that we can all be
vast-ass prize.

it hasn't said us
about the gutters
or the suicides. 

or the terror of one person
aching in one place
alone 

empty
unspoken to 

dishwater a plant. 

people are not glorious to each other.
people are not glorious to each other.
people are not glorious to each other. 

i suppose they never wi

# Part 2 : Topic Modeling and Summarization

## Summarization

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
def removeQuotes(mydata):
    new_data = mydata.strip('“”')
    return new_data
def SummaryOfPoem(poet):
  count = 0
  for key in poet['poems_array']:
    count += 1;
    text = poet[key]
    text = removeQuotes(text)
    summary_text = summarizer(text, max_length=50, min_length=5, do_sample=False)[0]['summary_text']
    poet["summary_"+key] = summary_text

  return poet

In [ ]:
def getSummary(file_name, df):
    summary = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            summary = json.load(file)
    else:
        summary = SummaryOfPoem(buildPoemArray(json.loads(df.to_json(orient="index"))))
        with open(file_name, 'w') as file:
            json.dump(summary, file)
    return summary

In [ ]:
file_name = store_loc + poet1.replace(" ", "_") + "-Summary.json"
WS_summary = getSummary(file_name, WS_df)
file_name = store_loc + poet2.replace(" ", "_") + "-Summary.json"
JWG_summary = getSummary(file_name, JWG_df)

## modifying library used for cosine similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

def getSemanticSimilarity_diff(embedding1, embedding2, model):
    if model == 1:
        return cosine_similarity([embedding1], [embedding2])[0][0]
    else:
        similarity = 1 - cosine(embedding1, embedding2)
        return similarity


In [ ]:
def swapWords_new(poet1, poet2, poem, cs_threshold, algo):
    model = SentenceTransformer('stsb-roberta-large')
    import time
    start = time.time()
    poet1_dict = get_poet_embedding_dict(poet1, model)
    end = time.time()
    start = time.time()
    poet2_dict = get_poet_embedding_dict(poet2, model)
    end = time.time()

    swapped_words_table = []

    # cs_threshold = 0.6
    new_poem = poem
    for word1 in poet1:
        max_word = ""
        max_sim = 0
        for word2 in poet2:
            if word1 == word2:
                continue
            sim = getSemanticSimilarity_diff(poet1_dict[word1], poet2_dict[word2], algo)
            if max_sim < sim:
                max_sim = sim
                max_word = word2
        if max_sim > cs_threshold:
            new_poem = new_poem.replace(word1, max_word)
            swapped_words_table.append([word1, max_word])
    return new_poem, swapped_words_table

In [ ]:
def newPoem_new(poet_1_pos, poet_2_pos, poem, index, cs_threshold, algo):
    new_poem = ""
    swapped_table = []
    new_poem, verbs_swapped = swapWords_new(poet_1_pos['verbs_poem' + str(index)], poet_2_pos['all_verbs'], poem[index].lower(), cs_threshold, algo)
    new_poem, adjectives_swapped = swapWords_new(poet_1_pos['adjectives_poem'+ str(index)], poet_2_pos['all_adjectives'], new_poem, cs_threshold, algo)
    new_poem, nouns_swapped = swapWords_new(poet_1_pos['nouns_poem'+ str(index)], poet_2_pos['all_nouns'], new_poem, cs_threshold, algo)
    swapped_table.extend(verbs_swapped)
    swapped_table.extend(adjectives_swapped)
    swapped_table.extend(nouns_swapped)
    return new_poem, swapped_table

In [ ]:
poem, table = newPoem_new(WS_pos, JWG_pos, WS_df.Poem_for_Human, 0, 0.6, 1)
getSemanticSimilarity(WS_df.Poem[0], poem, SentenceTransformer('stsb-roberta-large'))

In [ ]:
poem, table = newPoem_new(WS_pos, JWG_pos, WS_df.Poem_for_Human, 0, 0.6, 2)
getSemanticSimilarity(WS_df.Poem[0], poem, SentenceTransformer('stsb-roberta-large'))

## Topic Modeling

## LDA

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
!python3 -m spacy download en

In [ ]:
def choherence_perplexity_calculator(poem_list):
  data_words = list(sent_to_words(poem_list))
  print(data_words[:1])
  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
  trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)

  print(trigram_mod[bigram_mod[data_words[0]]])


  def remove_stopwords(texts):
      return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

  def make_bigrams(texts):
      return [bigram_mod[doc] for doc in texts]

  def make_trigrams(texts):
      return [trigram_mod[bigram_mod[doc]] for doc in texts]

  def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
      """https://spacy.io/api/annotation"""
      texts_out = []
      for sent in texts:
          doc = nlp(" ".join(sent))
          texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
      return texts_out
  # Remove Stop Words
  #data_words_nostops = remove_stopwords(data_words)

  # Form Bigrams
  data_words_bigrams = make_bigrams(data_words)

  # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
  # python3 -m spacy download en
  nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

  # Do lemmatization keeping only noun, adj, vb, adv
  data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN']) #, 'ADJ', 'VERB', 'ADV'

  print(data_lemmatized[:1])
  # Create Dictionary
  id2word = corpora.Dictionary(data_lemmatized)

  # Create Corpus
  texts = data_lemmatized

  # Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]

  # View
  print(corpus[:1])

  lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
  print(lda_model.print_topics())
  # Visualize the topics
  pyLDAvis.enable_notebook()
  vis = gensimvis.prepare(lda_model, corpus, id2word)
  # vis
  # Compute Perplexity
  print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

  # Compute Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nCoherence Score: ', coherence_lda)
  #return vis
  return lda_model, corpus, id2word

In [ ]:
WS_input = json.loads(WS_df.to_json(orient="index"))
JWG_input = json.loads(JWG_df.to_json(orient="index"))

In [ ]:
poem_count = len(WS_input)
WS_list=[]
for j in WS_input:
  poem_index =  str(j)
  WS_input[poem_index].pop('Poem_for_Human')
  WS_list.append(WS_input[poem_index])
print(WS_list)

In [ ]:
poem_count = len(JWG_input)
JWG_list=[]
for j in JWG_input:
  poem_index =  str(j)
  JWG_input[poem_index].pop('Poem_for_Human')
  JWG_list.append(JWG_input[poem_index])
print(JWG_list)

In [ ]:
lda_model, bow, diction = choherence_perplexity_calculator(WS_list)

In [ ]:
gensimvis.prepare(lda_model, bow, diction)

In [ ]:
lda_model, bow, diction = choherence_perplexity_calculator(JWG_list)

In [ ]:
gensimvis.prepare(lda_model, bow, diction)

In [ ]:
def ner(text):
  return spacy.load("en_core_web_sm")(text)

In [ ]:
original = ner(WS_input['0']['Poem'])

In [ ]:
displacy.render(original, style="ent", jupyter=True)

## BERTopic

In [ ]:
def remove_stopwords(texts):
    stopword_list = nltk.corpus.stopwords.words('english')
    return [[word for word in simple_preprocess(str(doc)) if word not in stopword_list] for doc in texts]

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(WS_df.Poem.to_list() + JWG_df.Poem.to_list()))

print(data_words[:1])
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
docs = WS_df.Poem.to_list() + JWG_df.Poem.to_list()

In [ ]:
docs = remove_stopwords(docs)
docs = make_bigrams(docs)
docs = make_trigrams(docs)
docs = lemmatization(docs)

In [ ]:
docs=[[' '.join(i)] for i in docs]
newDocs = list()
for i in docs:
    for j in i:
        newDocs.append(j)
print(newDocs)

In [ ]:
model = BERTopic(embedding_model=SentenceTransformer('stsb-roberta-large'))

topics, probabilities = model.fit_transform(newDocs)

In [ ]:
model.get_topic_freq().head(11)

In [ ]:
model.get_topic(-1)

## News scraping and Topic Modeling

In [ ]:
import urllib.request
from bs4 import BeautifulSoup as bs

def clean_txt(text):
    return ' '.join(text.split())

def scrapeNewsFromURL(url_list):
    news_results = []

    for url in url_list:
        header = {'User-Agent': 'Mozilla/5.0'}
        request = urllib.request.Request(url, headers=header)
        request_read = urllib.request.urlopen(request).read()

        soup = bs(request_read, 'html.parser')
        raw = soup.find('div', class_='article__content-container').text
        clean = clean_txt(raw)
        news_results.append(clean)

    return news_results

In [ ]:
NEWS_URLs = [
"https://www.cnn.com/2022/04/16/asia/north-korea-weapon-test-intl-hnk/index.html",
"https://www.cnn.com/2022/04/18/europe/ukraine-russia-mariupol-monday-intl/index.html",
"https://www.cnn.com/2022/04/08/americas/brazil-amazon-deforestation-latam-intl/index.html",
"https://www.cnn.com/2022/04/07/americas/colombia-landslide-miners-intl/index.html"
]

In [ ]:
NEWS_list=scrapeNewsFromURL(NEWS_URLs)

In [ ]:
lda_model, bow, diction = choherence_perplexity_calculator(NEWS_list)

In [ ]:
gensimvis.prepare(lda_model, bow, diction)